In [7]:
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [8]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAMM0aQEAAAAAtQeXJv5sh67MF%2BsrI3HrIvj5yIg%3D2G6yHNL05XqppqcA3fF4YwDr1ooIugZ1d5K6RrvnpfrKfHCCPr'

In [9]:
def auth():
    return os.getenv('TOKEN')

In [10]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [13]:
def create_url(keyword, start_date, end_date, max_results = 10):
    
    search_url = "https://api.twitter.com/2/tweets/search/all" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,geo.place_id',
                    'tweet.fields': 'author_id,created_at,id,geo,lang,public_metrics,text,withheld',
                    'user.fields': 'description,location,name,protected,username,verified',
                    'place.fields': 'contained_within,country,country_code,full_name,id,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)

In [14]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [105]:
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "dignity disability -is:retweet place_country:US"
start_time = '2017-01-01T00:00:00.00Z'
end_time = '2022-03-16T13:00:00.00Z'
max_results = 500

In [106]:
url = create_url(keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 200


In [107]:
with open('data.json', 'w') as f:
    json.dump(json_response, f)

In [108]:
df = pd.DataFrame(json_response['data'])
df2 = pd.DataFrame(json_response['includes']['users'])
df3 = pd.DataFrame(json_response['includes']['places'])

In [109]:
frames = [df, df2,df3]
result = pd.concat(frames, axis=1)

In [118]:
result.to_csv('data.csv')